# Model scaffold v0
Heavily based off of prototype

In [1]:
# Imports
from binance.client import Client
from binance.enums import * #https://github.com/sammchardy/python-binance/blob/master/binance/enums.py

# import schedule
import time
from datetime import datetime, timedelta

import statsmodels.formula.api as sm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# python files
from keys import key
from components import Downloader, Magic
import binance_helpers as bh

from IPython.display import clear_output

client = bh.new_binance_client()

mode = 2

In [2]:
dl = Downloader(client, "../data/model/")

In [3]:
df = dl.get_minutely_data("ICPUSDT")

In [4]:
dl.save_df(df, "ICPUSDT")

In [6]:
dl = Downloader(client, "../data/model/")
df_a = dl.get_minutely_data("FETUSDT")
dl.save_df(df, "FETUSDT")

In [7]:
dl = Downloader(client, "../data/model/")
df = dl.get_minutely_data("CELRUSDT")
dl.save_df(df, "CELRUSDT")

In [8]:
m = Magic(4000)

In [12]:
start = time.time()
df_b = dl.get_minutely_data("FETUSDT")
df_a = dl.get_minutely_data("CELRUSDT")
z = m.get_z_score(df_a, df_b)
# dl.save_df(df_b, "FETUDST")
# dl.save_df(df_a, "CELRUSDT")
time.time() - start

4.0957770347595215

In [4]:
df_a = dl.read_df("CELRUSDT")

In [23]:
m = Magic(4000)

In [11]:
start = time.time()
z = m.get_z_score(df_a, df_b)
time.time() - start

1.4940381050109863

In [25]:
z

-1.817757832127045

In [82]:
df_past = df[:-2]
df_now = df[-4:]

In [83]:
df = df_past[df_past.timestamp < df_now.iloc[0].timestamp]

In [84]:
df = df.append(df_now, ignore_index=True)

In [86]:
df.iloc[-1].timestamp

1627577640000.0

In [4]:
df.iloc[-1].timestamp

1627577640000.0

In [ ]:
def get_z_score(pair:str):
    """Returns the latest zscore + update or downloads all necessary files"""
    

In [15]:
datetime.utcfromtimestamp(1628050140000.0/1000)

datetime.datetime(2021, 8, 4, 4, 9)

In [5]:
datetime.utcnow()

datetime.datetime(2021, 8, 4, 2, 6, 20, 882577)

In [6]:
period = datetime.utcnow() - datetime.utcfromtimestamp(1627577640000.0/1000)

In [ ]:
datetime

In [59]:
math.ceil((period.days*24*60*60 + period.seconds)/60)

6208

In [70]:
period += timedelta(seconds=3600)

In [71]:
period

datetime.timedelta(days=4, seconds=34033, microseconds=768942)

In [27]:
date = datetime.utcfromtimestamp(1000000000000.0/1000)

In [28]:
date.strftime("%d %b %Y %H:%M:%S")

'09 Sep 2001 01:46:40'

In [4]:
datetime.strptime("1 Jan 1980", '%d %b %Y').strftime("%d %b %Y %H:%M:%S")

'01 Jan 1980 00:00:00'

In [38]:
test() if 3>4 else None

In [106]:
df = pd.DataFrame({"A":[0, 1], "B":[2, 3]})

In [107]:
def test(d):
    d = d.set_index("B")
    return d

In [111]:
df

,A,B
0,0,2
1,1,3


In [29]:
def test():
    print("Hi")

In [32]:
test() if 1>2 else print("bye")

bye


In [47]:
class test:
    def __init__(self, a):
        self.a = a
    def test1(self):
        return self.test2(self.a)
    def test2(self, a):
        return a

In [48]:
test = test(3)

In [49]:
test.test1()

3

In [24]:
client.KLINE_INTERVAL_1MINUTE

'1m'

In [18]:
start = time.time()
d = Downloader(client, "../data/")
time.time()-start

0.0001468658447265625

In [19]:
d.test()

../data/


In [2]:
bh.get_price(client, "BTCUSDT")

39668.85

In [3]:
bh.trade_amt(client, "BTC", 15000, 5)

0.37812

In [2]:
## Constants
ASSET_A = "CELR"
ASSET_B = "FET"
BASE = "USDT"
LOOKBACK = 4000 #1800
THRES = 1.
SELL_THRES = 1.

In [3]:
bh.get_margin_asset(client, ASSET_B)

{'asset': 'FET',
 'borrowEnabled': True,
 'borrowed': '0',
 'free': '0.14289781',
 'interest': '0',
 'locked': '0',
 'netAsset': '0.14289781',
 'netAssetOfBtc': '0.00000143',
 'repayEnabled': True,
 'totalAsset': '0.14289781'}

In [ ]:
def go_long_short(long: str, l_amt: float, short: str, s_amt: float, base=BASE):
    """goes long and short. Enter long/short as 'XMR' rather than 'XMRUSDT'. """
    l_pair = long + base
    s_pair = short + base
    transaction = client.create_margin_loan(asset=short, amount=str(s_amt), isIsolated='TRUE', symbol=s_pair)
#     time.sleep(1)
    print("starting short order")
    order_s = client.create_margin_order(
        symbol=s_pair,
        side=SIDE_SELL,
        type=ORDER_TYPE_MARKET,
        quantity=s_amt,
        newOrderRespType = "FULL",
        isIsolated='TRUE')
    print(order_s)
    print("starting buy order")
    order_l = client.create_margin_order(
        symbol=l_pair,
        side=SIDE_BUY,
        type=ORDER_TYPE_MARKET,
        quantity=l_amt,
        newOrderRespType = "FULL",
        isIsolated='TRUE')
    print(order_l)
    send_message("ls", long=long, l_amt=str(l_amt), short=short, s_amt=str(s_amt), o_s=order_s, o_l=order_l)
    
def liquidate(long: str, short: str, base=BASE):
    """liquidates long and short position. Enter long as currently longing which asset"""
    l_pair = long + base
    s_pair = short + base
    l_amt = binance_floor(float(get_margin_asset(long)["free"]), 1)
    s_amt = binance_ceil(abs(float(get_margin_asset(short)["netAsset"])), 1)
    
    order = client.create_margin_order(
        symbol=l_pair,
        side=SIDE_SELL,
        type=ORDER_TYPE_MARKET,
        quantity=l_amt,
        newOrderRespType = "FULL",
        isIsolated='TRUE')
    order1 = client.create_margin_order(
        symbol=s_pair,
        side=SIDE_BUY,
        type=ORDER_TYPE_MARKET,
        quantity=s_amt,
        newOrderRespType = "FULL",
        isIsolated='TRUE')
    rp = str(abs(float(get_margin_asset(short)["free"])))
    transaction1 = client.repay_margin_loan(asset=short, amount=rp, isIsolated='TRUE', symbol=s_pair)
    rebalance_accounts(ASSET_A, ASSET_B)
    send_message("lq", o_s=order, o_b=order1)
    
def rebalance_accounts(a: str, b: str, base=BASE):
    """Rebalance isolated margin accounts a and b. Enter as 'FET' """
    a_usdt = float(get_isolated_margin_account(a)["quoteAsset"]["free"])
    b_usdt = float(get_isolated_margin_account(b)["quoteAsset"]["free"])
    transfer = str(round(abs(a_usdt - (a_usdt + b_usdt)/2), 6))
    from_account = ""
    to_account = ""
    if a_usdt > b_usdt:
        from_account = a + base
        to_account = b + base
    else:
        from_account = b + base
        to_account = a + base
    client.transfer_isolated_margin_to_spot(asset='USDT', symbol=from_account, amount=transfer)
    client.transfer_spot_to_isolated_margin(asset='USDT', symbol=to_account, amount=transfer)

def get_long():
    """Returns None if long nothing, else ASSET_A or ASSET_B"""
    if float(get_margin_asset(ASSET_A)["borrowed"])>0: #Borrowed ASSET_A (ie long ASSET_B)
        return ASSET_B
    elif float(get_margin_asset(ASSET_B)["borrowed"])>0: #Borrwed XMR (ie long ZEC)
        return ASSET_A
    else:
        return None

In [ ]:
def printer():
    """main printer. Fetches current position, fetches data, and performs buys/sells if necessary"""
    long = get_long() # Returns None if long nothing, else ASSET_A or ASSET_B

    # Get z_score
    z = get_z_score()
    while math.isnan(z): #z is nan, retry:
        time.sleep(1)
        z = get_z_score()
    
    # Get trading amount (won't be used if liquidating or NOT buying)
    # NOTE: ASSUMES isolated margin account for both A and B have the same amount of USDT
    total = float(get_isolated_margin_account(ASSET_A)["quoteAsset"]["free"])*0.95
    asset_a = trade_amt(ASSET_A, total)
    asset_b = trade_amt(ASSET_B, total)
    
    if z>THRES and long==None: #buy/sell #LONG A
        go_long_short(long=ASSET_A, l_amt=asset_a, short=ASSET_B, s_amt=asset_b)
    elif z<-THRES and long==None: #buy/sell #LONG B
        go_long_short(long=ASSET_B, l_amt=asset_b, short=ASSET_A, s_amt=asset_a)
    elif z>SELL_THRES and long==ASSET_B: #liquidate #LONG B
        liquidate(long=ASSET_B, short=ASSET_A)
    elif z<-SELL_THRES and long==ASSET_A: #liquidate #LONG A
        liquidate(long=ASSET_A, short=ASSET_B)
    
    send_message("z", z=str(z))
    change_mode(z, long)
    reset_client()
    
    return schedule.CancelJob

In [ ]:
def change_mode(z:float, long):
    """Change the frequency of the checking"""
    global mode
    if long==None:
        mode = 2
    elif long==ASSET_A:
        if z > -SELL_THRES + 1.5: 
            mode = 10
        elif z > -SELL_THRES + 1.: 
            mode = 5
        elif z > -SELL_THRES + 0.5: 
            mode = 3
        else:
            mode = 2
    elif long==ASSET_B:
        if z < SELL_THRES - 1.5:
            mode = 10
        elif z < SELL_THRES - 1.:
            mode = 5
        elif z < SELL_THRES - .5:
            mode = 3
        else:
            mode = 2

In [ ]:
# Discord # Saving message
def get_message():
    """gets the message"""
    with open('message.txt') as json_file:
        new_message = json.load(json_file)
        return new_message
def send_message(m:str,  **k):
    data = get_message()
    if m == "ls":
        data["message"] = f"Went long {k['l_amt']} {k['long']} at ${k['o_l']['fills'][0]['price']} \nShort {k['s_amt']} {k['short']} at ${k['o_s']['fills'][0]['price']}"
        data["o_l"] = k['o_l']
        data["o_s"] = k['o_s']
    elif m == "lq":
        o_s = k['o_s']
        o_b = k['o_b']
        data["message"] = f"Liquidated! Sold {o_s['executedQty']} {o_s['symbol']} for {o_s['fills'][0]['price']} \nBought {o_b['executedQty']} {o_b['symbol']} for {o_b['fills'][0]['price']}"
        data["o_l"] = o_s
        data["o_s"] = o_b
    elif m == "z":
        data["z"] = k["z"]
    elif m == "mode":
        data["mode"] = k["mode"]
    with open('message.txt', 'w') as outfile:
        json.dump(data, outfile)